In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split

# Building the model

In [2]:
# load data and split it into Test, train and validation sets

(X_1, y_1), (X_2, y_2) = tf.keras.datasets.cifar10.load_data()
X = np.vstack((X_1, X_2))
y = np.vstack((y_1, y_2))
X_train, X, y_train, y = train_test_split(X, y, train_size=0.8, stratify=y)
X_test, X_valid, y_test, y_valid = train_test_split(X, y, test_size=0.5, stratify=y)

assert X_test.shape == X_valid.shape, 'X_test and X_valid have different shapes'
for i in range(10):
    assert np.count_nonzero(keras.utils.to_categorical(y_train)[:, i]) == 4800 , 'Sample is not stratified'
sample_shape = X_train[0].shape
print(f'Shape of a single sample: {sample_shape}')

Shape of a single sample: (32, 32, 3)


In [7]:
# build the model

model = Sequential()
model.add(Input(shape=sample_shape))
model.add(Conv2D(8, kernel_size = (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(12, kernel_size = (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(16, kernel_size = (3,3), padding='same', activation='relu'))
model.add(UpSampling2D())
model.add(Conv2D(12,kernel_size = (3,3), padding='same', activation='relu'))
model.add(UpSampling2D(size = (2,2)))
model.add(Conv2D(3,(3,3), padding='same', activation='relu'))


# compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError(),
            keras.metrics.CosineSimilarity()])
# model.build(input_shape=(4800,32,32,1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 32, 32, 8)         224       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 12)        876       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 12)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 16)          1744      
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 16, 12)       

In [8]:
# train the model
history = model.fit(X_train, X_train, epochs=10, verbose=1, use_multiprocessing=True)

Epoch 1/10
1500/1500 [==============================] - 48s 31ms/step - loss: 731.0197 - mean_absolute_error: 18.9471 - cosine_similarity: 0.9868
Epoch 2/10
1500/1500 [==============================] - 48s 32ms/step - loss: 399.6917 - mean_absolute_error: 14.3238 - cosine_similarity: 0.9907
Epoch 3/10
1500/1500 [==============================] - 50s 34ms/step - loss: 358.0405 - mean_absolute_error: 13.4980 - cosine_similarity: 0.9908
Epoch 4/10
1500/1500 [==============================] - 50s 33ms/step - loss: 334.2354 - mean_absolute_error: 13.0176 - cosine_similarity: 0.9908
Epoch 5/10
1500/1500 [==============================] - 51s 34ms/step - loss: 317.3307 - mean_absolute_error: 12.6550 - cosine_similarity: 0.9908
Epoch 6/10
1500/1500 [==============================] - 50s 33ms/step - loss: 304.5045 - mean_absolute_error: 12.3768 - cosine_similarity: 0.9909
Epoch 7/10
1500/1500 [==============================] - 50s 33ms/step - loss: 293.6784 - mean_absolute_error: 12.1414 - cosi

In [15]:
# save trained model

def save_model(model, path='models/CAE'):
    model.save(path)

save_model(model)

INFO:tensorflow:Assets written to: models/CAE/assets


In [13]:
# load a pretrained model

def load_model(path = 'models/CAE'):
    return keras.models.load_model(path)


pretrained_model = load_model()